# XGBoost による顧客のチャーン予測
_**勾配ブースティング木を使ったモバイル顧客の離脱率予測**_

---

---

## Contents

1. [Background](#Background)
1. [Setup](#Setup)
1. [Data](#Data)
1. [Train](#Train)
1. [Host](#Host)
  1. [Evaluate](#Evaluate)
  1. [Relative cost of errors](#Relative-cost-of-errors)
1. [Extensions](#Extensions)

---

## Background

_この Notebook は [AWS blog post](https://aws.amazon.com/blogs/ai/predicting-customer-churn-with-amazon-machine-learning/) とそれに付随する SageMaker Examples の和訳です_

顧客を失うことはビジネスでは高く付きます。満足度の低い顧客を早い段階で特定することで、利用継続のインセンティブを与えられる可能性があります。この Notebook は機械学習 (ML) を用いて満足度の低い顧客を自動的に特定する方法 -- 顧客のチャーン予測 (customer churn prediction) とも呼ばれます -- を説明します。ML モデルが完璧な予測をすることはまれなので、この Notebook では ML を利用する経済的な結果を決める際の予測ミスの相対的なコストをどう取り入れるかについても書きます。

ここでは我々にとって身近なチャーン、携帯電話事業者を解約する例を用いることにします。(だって、不満ならいくらでも見つけられるでしょう。) もし通信会社が自分が解約しようとしていることを知っていれば、一時的なインセンティブ -- いつでも携帯をアップグレードできるとか、新しい機能が使えるようになるとか -- を与えて契約を継続させるでしょう。インセンティブは通常、顧客を再獲得するよりも圧倒的にコスト効率が良いのです。

Losing customers is costly for any business.  Identifying unhappy customers early on gives you a chance to offer them incentives to stay.  This notebook describes using machine learning (ML) for the automated identification of unhappy customers, also known as customer churn prediction. ML models rarely give perfect predictions though, so this notebook is also about how to incorporate the relative costs of prediction mistakes when determining the financial outcome of using ML.

We use an example of churn that is familiar to all of us–leaving a mobile phone operator.  Seems like I can always find fault with my provider du jour! And if my provider knows that I’m thinking of leaving, it can offer timely incentives–I can always use a phone upgrade or perhaps have a new feature activated–and I might just stick around. Incentives are often much more cost effective than losing and reacquiring a customer.

---

## Setup

_このノートブックの作成およびテストは ml.m4.xlarge ノートブックインスタンスを用いて行われました。_

以下のものを用意して始めましょう: 
- トレーニングとモデルデータの置き場所として使う S3 バケットとプレフィックス。ノートブックインスタンス・学習・デプロイ場所と同じリージョンにある必要があります。
- 学習・デプロイ用のコンテナがデータにアクセスするための IAM ロール ARN。これらの作成方法については[ドキュメント](https://docs.aws.amazon.com/ja_jp/IAM/latest/UserGuide/id_roles_create_for-service.html)を参照して下さい。**[注意]** もしノートブックインスタンス、学習、デプロイの際に2つ以上のロールが必要な場合、boto regexp を適切な IAM ロール ARN 文字列で置き換えて下さい。

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

In [ ]:
bucket = '<your_s3_bucket_name_here>'
prefix = 'sagemaker/DEMO-xgboost-churn'

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

次に、必要な Python ライブラリを import します。

Next, we'll import the Python libraries we'll need for the remainder of the exercise.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
import sagemaker
from sagemaker.predictor import csv_serializer

---
## Data

携帯電話事業者はどの顧客が解約し、誰がサービスを継続利用しているかの履歴データを持っています。この過去の情報を使って ML モデルを学習させることにより、ある事業者の解約率を予測するモデルを構築することができます。モデルを学習させた後、任意の顧客情報 (学習に使った情報と同様のもの) をモデルに入力することによって、顧客が解約しようとしているかを予測することができます。もちろん、モデルが予測を間違えることがあるため -- 結局のところ、未来を予測することは一筋縄ではいかないということなのです。しかし、ここではその予測誤差とどう付き合っていくかについても言及します。

今回扱うのは、Daniel T. Larose の [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) という本で述べられている公開データセットです。これは University of California Irvine (UCI) 機械学習レポジトリの著者に帰属します。さて、ダウンロードしてデータセットを見てみましょう: 

Mobile operators have historical records on which customers ultimately ended up churning and which continued using the service. We can use this historical information to construct an ML model of one mobile operator’s churn using a process called training. After training the model, we can pass the profile information of an arbitrary customer (the same profile information that we used to train the model) to the model, and have the model predict whether this customer is going to churn. Of course, we expect the model to make mistakes–after all, predicting the future is tricky business! But I’ll also show how to deal with prediction errors.

The dataset we use is publicly available and was mentioned in the book [Discovering Knowledge in Data](https://www.amazon.com/dp/0470908742/) by Daniel T. Larose. It is attributed by the author to the University of California Irvine Repository of Machine Learning Datasets.  Let's download and read that dataset in now:

In [ ]:
!wget http://dataminingconsultant.com/DKD2e_data_sets.zip
!unzip -o DKD2e_data_sets.zip

In [ ]:
churn = pd.read_csv('./Data sets/churn.txt')
pd.set_option('display.max_columns', 500)
churn

最近の基準からすると比較的小さなデータセットで、3,333 レコードと、それぞれ 21 個の属性が、とある US の携帯事業者の顧客データを表しています。それぞれの属性は: 
- `State`: 顧客が住んでる US の州で、2文字の略号。例: OH, NJ
- `Account Length`: このアカウントがアクティブだった日数
- `Area Code`: 顧客の電話番号に対応する3桁の地域コード
- `Phone`: のこりの7桁の電話番号
- `Int’l Plan`: 顧客が国際電話プランに契約しているかどうか: yes/no
- `VMail Plan`: 顧客がボイスメール機能を使っているかどうか: yes/no
- `VMail Message`: (恐らく) 月ごとのボイスメールの平均メッセージ数
- `Day Mins`: 日中に使用された通話時間の合計数
- `Day Calls`: 日中にかけられた電話の回数
- `Day Charge`: 昼間の通話料金
- `Eve Mins, Eve Calls, Eve Charge`: 夕方の通話料金
- `Night Mins`, `Night Calls`, `Night Charge`: 夜間の通話料金
- `Intl Mins`, `Intl Calls`, `Intl Charge`: 国際電話の通話料金
- `CustServ Calls`: カスタマーサービスへの通話数
- `Churn?`: 顧客が解約したかどうか: true/false

最後の属性 `Churn?` はターゲット属性として知られています -- 我々が ML モデルに予測してほしい属性です。ターゲット属性が2値なので、我々のモデルは2値の予測 (2値分類) を行うよう設計します。

データを見てみましょう: 

By modern standards, it’s a relatively small dataset, with only 3,333 records, where each record uses 21 attributes to describe the profile of a customer of an unknown US mobile operator. The attributes are:

- `State`: the US state in which the customer resides, indicated by a two-letter abbreviation; for example, OH or NJ
- `Account Length`: the number of days that this account has been active
- `Area Code`: the three-digit area code of the corresponding customer’s phone number
- `Phone`: the remaining seven-digit phone number
- `Int’l Plan`: whether the customer has an international calling plan: yes/no
- `VMail Plan`: whether the customer has a voice mail feature: yes/no
- `VMail Message`: presumably the average number of voice mail messages per month
- `Day Mins`: the total number of calling minutes used during the day
- `Day Calls`: the total number of calls placed during the day
- `Day Charge`: the billed cost of daytime calls
- `Eve Mins, Eve Calls, Eve Charge`: the billed cost for calls placed during the evening
- `Night Mins`, `Night Calls`, `Night Charge`: the billed cost for calls placed during nighttime
- `Intl Mins`, `Intl Calls`, `Intl Charge`: the billed cost for international calls
- `CustServ Calls`: the number of calls placed to Customer Service
- `Churn?`: whether the customer left the service: true/false

The last attribute, `Churn?`, is known as the target attribute–the attribute that we want the ML model to predict.  Because the target attribute is binary, our model will be performing binary prediction, also known as binary classification.

Let's begin exploring the data:

In [ ]:
# Frequency tables for each categorical feature
for column in churn.select_dtypes(include=['object']).columns:
    display(pd.crosstab(index=churn[column], columns='% observations', normalize='columns'))

# Histograms for each numeric features
display(churn.describe())
%matplotlib inline
hist = churn.hist(bins=30, sharey=True, figsize=(10, 10))

すぐに以下のことが分かります: 
- `State` は一様に分布している
- `Phone` は実用的でない大量の一意な値を取る。プレフィックスを使うことはできるかもしれないが、どういう割当がされてるかが分からないならこれは使わないほうがよい
- 顧客の 14% だけが解約している。2クラス間のデータ数に不均衡があるが、それほど極端ではない
- ほとんどの数値特徴量は驚くほどいい感じに分布していて、釣り鐘型 -- ガウシアン的な分布をしている。`VMail Message` は特筆すべき例外 (そして `Area Code` は非数値型に変換すべき)

We can see immediately that:
- `State` appears to be quite evenly distributed
- `Phone` takes on too many unique values to be of any practical use.  It's possible parsing out the prefix could have some value, but without more context on how these are allocated, we should avoid using it.
- Only 14% of customers churned, so there is some class imabalance, but nothing extreme.
- Most of the numeric features are surprisingly nicely distributed, with many showing bell-like gaussianity.  `VMail Message` being a notable exception (and `Area Code` showing up as a feature we should convert to non-numeric).

In [ ]:
churn = churn.drop('Phone', axis=1)
churn['Area Code'] = churn['Area Code'].astype(object)

次はそれぞれの特徴量とターゲット間の関係を見てみましょう。

Next let's look at the relationship between each of the features and our target variable.

In [ ]:
for column in churn.select_dtypes(include=['object']).columns:
    if column != 'Churn?':
        display(pd.crosstab(index=churn[column], columns=churn['Churn?'], normalize='columns'))

for column in churn.select_dtypes(exclude=['object']).columns:
    print(column)
    hist = churn[[column, 'Churn?']].hist(by='Churn?', bins=30)
    plt.show()

面白いことにチャーンは: 
- 地理的に均等に分布している
- 国際プランに入っている傾向がある
- ボイスメールには入らない傾向にある
- 日毎の利用時間が二峰性になっている (非解約者に比べて高いか低いかのどちらか)
- 多くの顧客がカスタマーサービスを使っている (多くの問題を経験した顧客が解約しやすい、というのは理解できる)

これに加えて、解約者は `Day Mins` や `Day Charge` のような特徴量について非常に似通った分布をしていることがわかります。通話時間は金額と相関するため特に驚きはない。それでは特徴量の関係についてもう少し深く見てみましょう。

Interestingly we see that churners appear:
- Fairly evenly distributed geographically
- More likely to have an international plan
- Less likely to have a voicemail plan
- To exhibit some bimodality in daily minutes (either higher or lower than the average for non-churners)
- To have a larger number of customer service calls (which makes sense as we'd expect customers who experience lots of problems may be more likely to churn)

In addition, we see that churners take on very similar distributions for features like `Day Mins` and `Day Charge`.  That's not surprising as we'd expect minutes spent talking to correlate with charges.  Let's dig deeper into the relationships between our features.

In [ ]:
display(churn.corr())
pd.plotting.scatter_matrix(churn, figsize=(12, 12))
plt.show()

いくつかの特徴量は本質的に 100% の相関を持っていることが分かります。これらの特徴量ペアを含め、マイナーな冗長性やバイアスとして扱われるものもありますが、いくつかの機械学習のアルゴリズムには致命的な問題となる可能性があります。それぞれから高い相関を持つペアを取り除いてみましょう。Day Mins から Day Charge を、Night Mins から Night Charge を、Intl Mins から Intl Charge を除外します:

We see several features that essentially have 100% correlation with one another.  Including these feature pairs in some machine learning algorithms can create catastrophic problems, while in others it will only introduce minor redundancy and bias.  Let's remove one feature from each of the highly correlated pairs: Day Charge from the pair with Day Mins, Night Charge from the pair with Night Mins, Intl Charge from the pair with Intl Mins:

In [ ]:
churn = churn.drop(['Day Charge', 'Eve Charge', 'Night Charge', 'Intl Charge'], axis=1)

さてこれでデータセットの前処理が終わったので、どのアルゴリズムを使うかを決定しましょう。上で述べたとおり、特定の値が高低どちらか (中間ではなく) にあればチャーンと予測できそうなことが分かっています。これを線形回帰などのアルゴリズムに入れるためには、多項式の (bucketed) 項を作る必要があります。かわりにこの問題を勾配ブースティング木を使ってモデル化することを考えましょう。Amazon SageMaker はマネージドで、分散学習でき、リアルタイム推論エンドポイントをデプロイできる XGBoost コンテナを提供しています。XGBoost は、特徴量とターゲット変数間の非線形な関係を自然に説明し、特徴量間の複雑な関係も記述する勾配ブースティング木を用いています。

Amazon SageMaker XGBoost は CSV あるいは LibSVM フォーマットどちらのデータでも学習できます。この例では CSV にこだわります。データは
- 予測変数を1列目にもつ
- ヘッダ行をもたない

必要があります。まずはじめにカテゴリ変数を数値に変換しましょう。

Now that we've cleaned up our dataset, let's determine which algorithm to use.  As mentioned above, there appear to be some variables where both high and low (but not intermediate) values are predictive of churn.  In order to accommodate this in an algorithm like linear regression, we'd need to generate polynomial (or bucketed) terms.  Instead, let's attempt to model this problem using gradient boosted trees.  Amazon SageMaker provides an XGBoost container that we can use to train in a managed, distributed setting, and then host as a real-time prediction endpoint.  XGBoost uses gradient boosted trees which naturally account for non-linear relationships between features and the target variable, as well as accommodating complex interactions between features.

Amazon SageMaker XGBoost can train on data in either a CSV or LibSVM format.  For this example, we'll stick with CSV.  It should:
- Have the predictor variable in the first column
- Not have a header row

But first, let's convert our categorical features into numeric features.

In [ ]:
model_data = pd.get_dummies(churn)
model_data = pd.concat([model_data['Churn?_True.'], model_data.drop(['Churn?_False.', 'Churn?_True.'], axis=1)], axis=1)

そして、データを training, validation, test セットに分けましょう。これによりモデルの過学習を防ぐことができ、未知のデータを用いてモデルの精度をテストすることができます。

And now let's split the data into training, validation, and test sets.  This will help prevent us from overfitting the model, and allow us to test the models accuracy on data it hasn't already seen.

In [ ]:
train_data, validation_data, test_data = np.split(model_data.sample(frac=1, random_state=1729), [int(0.7 * len(model_data)), int(0.9 * len(model_data))])
train_data.to_csv('train.csv', header=False, index=False)
validation_data.to_csv('validation.csv', header=False, index=False)

これらのファイルを S3 にアップロードします。

Now we'll upload these files to S3.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')

---
## Train

トレーニングに移ります。まず XGBoost アルゴリズムコンテナの場所を指定する必要があります。

Moving onto training, first we'll need to specify the locations of the XGBoost algorithm containers.

In [ ]:
from sagemaker.amazon.amazon_estimator import get_image_uri
container = get_image_uri(boto3.Session().region_name, 'xgboost')

次に、今回 CSV ファイルを使ってトレーニングを行うので、S3 に置いたファイルのポインターとして学習用の関数が使う `s3_input` を作ります。

Then, because we're training with the CSV file format, we'll create `s3_input`s that our training function can use as a pointer to the files in S3.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

ここで、いくつかのパラメータ -- 学習に使うインスタンスの種類と数、XGBoost のハイパーパラメータ -- を指定します。いくつかの重要なパラメータは: 
- `max_depth` はアルゴリズム内でそれぞれの木がどれくらい深く作られるかをコントロールします。木が深いほどフィッティングは良くなりますが、計算量が多くなり過学習しやすくもなります。典型的にはモデルのパフォーマンスとトレードオフがあり、多数の浅い木と少数の深い木の間でパラメータを探索する必要があります。
- `subsample` は学習データのサンプリングをコントロールします。これは過学習を防ぎますが、あまり低くしすぎるとデータ不足になります。
- `num_round` はブースティングのラウンド数をコントロールします。これは本質的には前のイテレーションの残差を使って引き続きモデルを学習させます。これも、ラウンドを増やせば学習データに対するフィッティングは良くなりますが、計算量が増えたり過学習する可能性があります。
- `eta` は各ブースティングラウンドのアグレッシブさを決定します。値が大きい方が保守的なブースティングになります。
- `gamma` は木の成長がどれだけアグレッシブ化を決めます。大きい値のほうが保守的なモデルを作ります。

詳細は GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.rst) を読んで XGBoost のハイパーパラメータを確認して下さい。

Now, we can specify a few parameters like what type of training instances we'd like to use and how many, as well as our XGBoost hyperparameters.  A few key hyperparameters are:
- `max_depth` controls how deep each tree within the algorithm can be built.  Deeper trees can lead to better fit, but are more computationally expensive and can lead to overfitting.  There is typically some trade-off in model performance that needs to be explored between a large number of shallow trees and a smaller number of deeper trees.
- `subsample` controls sampling of the training data.  This technique can help reduce overfitting, but setting it too low can also starve the model of data.
- `num_round` controls the number of boosting rounds.  This is essentially the subsequent models that are trained using the residuals of previous iterations.  Again, more rounds should produce a better fit on the training data, but can be computationally expensive or lead to overfitting.
- `eta` controls how aggressive each round of boosting is.  Larger values lead to more conservative boosting.
- `gamma` controls how aggressively trees are grown.  Larger values lead to more conservative models.

More detail on XGBoost's hyperparmeters can be found on their GitHub [page](https://github.com/dmlc/xgboost/blob/master/doc/parameter.md).

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)
xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

---
## Host

それではモデルを学習させたので、エンドポイントにデプロイしましょう。

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

### Evaluate

これでエンドポイントは動いているので、http POST リクエストを投げることで、リアルタイム推論を非常に簡単に行うことができます。しかしまず、`test_data` の NumPy array をエンドポイントの裏のモデルに渡すために、シリアライザーとデシリアライザーを設定しなければなりません。

Now that we have a hosted endpoint running, we can make real-time predictions from our model very easily, simply by making an http POST request.  But first, we'll need to setup serializers and deserializers for passing our `test_data` NumPy arrays to the model behind the endpoint.

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = None

以下の機能を持った簡単な関数を作ります: 
1. test データセットをループする
1. ミニバッチに分割する
1. CSV string payload に変換する
1. XGBoost エンドポイントを呼び出してミニバッチに対する予測を行う
1. 予測値を集めてモデルから出力された CSV を NumPy array に変換する

Now, we'll use a simple function to:
1. Loop over our test dataset
1. Split it into mini-batches of rows 
1. Convert those mini-batchs to CSV string payloads
1. Retrieve mini-batch predictions by invoking the XGBoost endpoint
1. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

predictions = predict(test_data.as_matrix()[:, 1:])

機械学習モデルのパフォーマンスを比較する多くの方法がありますが、単純に正しい値と予測値を比べてみましょう。この場合、単純に顧客が解約する (`1`) か、しない (`0`) かを予測し、混同行列が得られます。

There are many ways to compare the performance of a machine learning model, but let's start by simply by comparing actual to predicted values.  In this case, we're simply predicting whether the customer churned (`1`) or not (`0`), which produces a simple confusion matrix.

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.round(predictions), rownames=['actual'], colnames=['predictions'])

_[注意] アルゴリズムの乱択性により、結果が少し違って見えるかもしれません。_

チャーンした人 48人のうち、39人を正しく予測することができました (true positive)。間違えて予測された4人はそのまま契約を続けるでしょう (false positive)。そのほか解約しないと予測された9人の顧客は実際にチャーンしています (false negative)。

ここで重要な点は `np.round()` 関数が原因で0.5という単純な閾値 (カットオフ値) を設定していることです。`xgboost` からの予測は0から1の間の連続値を取るので、それを元々の2クラスに戻して解釈しています。しかし、離脱する顧客は、離脱しようとして企業がより積極的に引き留めようとする顧客よりもコストがかかることが期待されるので、このカットオフを調整する必要があります。これはほとんど確実に false positive を増やしますが、同時にtrue positive も増やし、false negative を減らします。

ざっくりとした直観を得るために、予測結果の連続量を見てみましょう。

_Note, due to randomized elements of the algorithm, you results may differ slightly._

Of the 48 churners, we've correctly predicted 39 of them (true positives). And, we incorrectly predicted 4 customers would churn who then ended up not doing so (false positives).  There are also 9 customers who ended up churning, that we predicted would not (false negatives).

An important point here is that because of the `np.round()` function above we are using a simple threshold (or cutoff) of 0.5.  Our predictions from `xgboost` come out as continuous values between 0 and 1 and we force them into the binary classes that we began with.  However, because a customer that churns is expected to cost the company more than proactively trying to retain a customer who we think might churn, we should consider adjusting this cutoff.  That will almost certainly increase the number of false positives, but it can also be expected to increase the number of true positives and reduce the number of false negatives.

To get a rough intuition here, let's look at the continuous values of our predictions.

In [ ]:
plt.hist(predictions)
plt.show()

モデルから出力された連続値は0と1の間ですが、0.1と0.9の間に十分な量があり、cutoffは予測される顧客数を変化させます。例えば、

The continuous valued predictions coming from our model tend to skew toward 0 or 1, but there is sufficient mass between 0.1 and 0.9 that adjusting the cutoff should indeed shift a number of customers' predictions.  For example...

In [ ]:
pd.crosstab(index=test_data.iloc[:, 0], columns=np.where(predictions > 0.3, 1, 0))

カットオフ値を0.5から0.3に変化させることで、もう1人の true positive と 3人の false positive、そして false negative が1人少なくなったことが分かります。数字は全体的に少ないですが、カットオフの変化により顧客の 6-10% に影響を与えています。これは正しい判断なのでしょうか？3人の顧客を繋ぎ止められますが、同時に5人に不要なインセンティブを与えていることになります。最適なカットオフを決めることは実世界に機械学習を応用する上で重要なステップになります。もう少し一般的に議論して、特定の、仮想的な解を適用しましょう。

We can see that changing the cutoff from 0.5 to 0.3 results in 1 more true positives, 3 more false positives, and 1 fewer false negatives.  The numbers are small overall here, but that's 6-10% of customers overall that are shifting because of a change to the cutoff.  Was this the right decision?  We may end up retaining 3 extra customers, but we also unnecessarily incentivized 5 more customers who would have stayed.  Determining optimal cutoffs is a key step in properly applying machine learning in a real-world setting.  Let's discuss this more broadly and then apply a specific, hypothetical solution for our current problem.

### Relative cost of errors

どんな2値分類問題も同じような感度のカットオフを生むことはありません。これはこれ自体では特に問題ではないです。結局、もし2クラスのスコアが十分簡単に分離可能なら、問題はそれほど難しくなく、ML ではなく単純なルールで問題を解くことが可能です。

これより重要なのはもし ML モデルで推論させても、モデルが間違って false positive と false negative を割り振るコストがあることです。同じように、true positives と true negatives の正しい推論に付随したコストも見る必要があります。なぜなら、カットオフの選び方はこれらの統計量の4つ全てに関わるからで、各推論に対して4つの出力がどのようなビジネス上の相対コストを生むか考慮する必要があるからです。

Any practical binary classification problem is likely to produce a similarly sensitive cutoff. That by itself isn’t a problem. After all, if the scores for two classes are really easy to separate, the problem probably isn’t very hard to begin with and might even be solvable with simple rules instead of ML.

More important, if I put an ML model into production, there are costs associated with the model erroneously assigning false positives and false negatives. I also need to look at similar costs associated with correct predictions of true positives and true negatives.  Because the choice of the cutoff affects all four of these statistics, I need to consider the relative costs to the business for each of these four outcomes for each prediction.

#### Assigning costs

今回問題にした携帯事業者の解約におけるコストとは何でしょうか? もちろんコストはビジネス上で取りうる具体的なアクションに依存します。ここでいくつかの仮定を置いてみましょう。

はじめに、true negative のコストを ¥0 とします。我々のモデルは本質的に、この場合幸せな顧客正しく特定できていることになるので、何もする必要はありません。

次に false negative は一番問題で、離脱しそうな顧客が留まると間違って予測するからです。顧客を失い、放棄所得、広告コスト、管理コスト、店頭コスト、そして恐らく携帯電話ハードウェア補助金を含む、代わりの顧客を獲得するコストを支払う必要があります。

最後に、解約しそうとモデルが予測した顧客については、リテンションのためのインセンティブが仮に一万円だとしましょう。もし事業会社が自分にこういう譲歩をしてきたら、さすがに解約までにもう一回考え直すでしょう。これが true positive と false positive の結果に対するコストです。ここで false posivie (顧客は満足だが誤って離脱と予測) の場合、一万円を _ドブに捨てる_ ことになります。恐らくこの一万円をもっと賢く使うことはできたでしょうが、既存の顧客の支持を高める可能性もあるため、それほど悪い選択ではありません。

What are the costs for our problem of mobile operator churn? The costs, of course, depend on the specific actions that the business takes. Let's make some assumptions here.

First, assign the true negatives the cost of \$0. Our model essentially correctly identified a happy customer in this case, and we don’t need to do anything.

False negatives are the most problematic, because they incorrectly predict that a churning customer will stay. We lose the customer and will have to pay all the costs of acquiring a replacement customer, including foregone revenue, advertising costs, administrative costs, point of sale costs, and likely a phone hardware subsidy. A quick search on the Internet reveals that such costs typically run in the hundreds of dollars so, for the purposes of this example, let's assume \$500. This is the cost of false negatives.

Finally, for customers that our model identifies as churning, let's assume a retention incentive in the amount of \$100. If my provider offered me such a concession, I’d certainly think twice before leaving. This is the cost of both true positive and false positive outcomes. In the case of false positives (the customer is happy, but the model mistakenly predicted churn), we will "waste" the \$100 concession. We probably could have spent that \$100 more effectively, but it's possible we increased the loyalty of an already loyal customer, so that’s not so bad.

#### Finding the optimal cutoff

false negatives が false positives よりも相当コストがかかるのは明らかです。顧客数をもとにエラーを最適化するかわりに、このようなコスト関数を最小化しましょう: 

```txt
$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)
```

ここで `FN(C)` は false negative がカットオフ `C` の関数であることを意味し、`TN, FP, TP` についても同様です。上の式の結果を最小化するカットオフ `C` を探す必要があります。

これを行う素直な方法は、複数の考えられるカットオフについてシミュレーションを走らせることです。以下では100通りのカットオフについてforループで計算を行います。


It’s clear that false negatives are substantially more costly than false positives. Instead of optimizing for error based on the number of customers, we should be minimizing a cost function that looks like this:

```txt
$500 * FN(C) + $0 * TN(C) + $100 * FP(C) + $100 * TP(C)
```

FN(C) means that the false negative percentage is a function of the cutoff, C, and similar for TN, FP, and TP.  We need to find the cutoff, C, where the result of the expression is smallest.

A straightforward way to do this, is to simply run a simulation over a large number of possible cutoffs.  We test 100 possible values in the for loop below.

In [ ]:
cutoffs = np.arange(0.01, 1, 0.01)
costs = []
for c in cutoffs:
    costs.append(np.sum(np.sum(np.array([[0, 100], [500, 100]]) * 
                               pd.crosstab(index=test_data.iloc[:, 0], 
                                           columns=np.where(predictions > c, 1, 0)))))

costs = np.array(costs)
plt.plot(cutoffs, costs)
plt.show()
print('Cost is minimized near a cutoff of:', cutoffs[np.argmin(costs)], 'for a cost of:', np.min(costs))

上の図は、閾値を低くしすぎると全顧客に対してリテンションインセンティブを渡すことになりコストが跳ね上がることを示しています。一方で、閾値を高くしすぎるとあまりに多くの顧客を手放すことになり最終的に同じぐらいコストがかかります。全体のコストはカットオフが 0.46 のときに最小の 84万円 になり、何もしない場合に 200万円 以上失うのよりは圧倒的に良いという結果になります。

The above chart shows how picking a threshold too low results in costs skyrocketing as all customers are given a retention incentive.  Meanwhile, setting the threshold too high results in too many lost customers, which ultimately grows to be nearly as costly.  The overall cost can be minimized at \$8400 by setting the cutoff to 0.46, which is substantially better than the \$20k+ I would expect to lose by not taking any action.

---
## Extensions

このノートブックでは顧客が離脱するのを予測するモデルを構築する方法と、true/false positive と false negative により生じるコストを最適化する閾値の決め方について披露しました。これを拡張するにはいくつかの方法が考えられます: 
- リテンションインセンティブを受け取る顧客も離脱の可能性がある。インセンティブを受け取っても解約する確率を含めるとリテンションプログラムのROIが向上する。
- 低価格のプランに移行したり課金オプションを無効化する顧客は別の種類のチャーンとしてモデル化できる。
- 顧客行動の発展をモデル化する。もし使用量が落ちてカスタマーサービスへの連絡回数が増えている場合、解約される可能性が高い。顧客情報は行動傾向を取り入れるべきである。
- 実際の学習データと金銭的コストはもっと複雑になり得る。
- それぞれのチャーンに合わせた複数のモデルが必要。

これらの複雑さに関わらず、このノートブックで説明したものと似たような原理は適用できるはずである。

This notebook showcased how to build a model that predicts whether a customer is likely to churn, and then how to optimally set a threshold that accounts for the cost of true positives, false positives, and false negatives.  There are several means of extending it including:
- Some customers who receive retention incentives will still churn.  Including a probability of churning despite receiving an incentive in our cost function would provide a better ROI on our retention programs.
- Customers who switch to a lower-priced plan or who deactivate a paid feature represent different kinds of churn that could be modeled separately.
- Modeling the evolution of customer behavior. If usage is dropping and the number of calls placed to Customer Service is increasing, you are more likely to experience churn then if the trend is the opposite. A customer profile should incorporate behavior trends.
- Actual training data and monetary cost assignments could be more complex.
- Multiple models for each type of churn could be needed.

Regardless of additional complexity, similar principles described in this notebook are likely apply.

### (Optional) Clean-up

このノートブックによって作られたリソースを削除していい場合、以下のセルを実行してください。このコマンドは上で作成したエンドポイントを削除して意図しない請求を防ぐことができます。
(必要であれば、このノートブック自体を走らせているノートブックインスタンスも SageMaker のマネージメントコンソールから停止させて下さい。)

If you're ready to be done with this notebook, please run the cell below.  This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)